In [28]:
# Comment out these lines
import sys
sys.path.insert(0, 'C:\\Users\\masch\\QuantumComputing\\QComp\\pgmpy')

# Imports
import cmath
import math
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.inference import BeliefPropagation

gamma = 0.7
theta = 2*math.asin(np.sqrt(gamma))

amp_damping = BayesianNetwork([('q1m0','q1m1'),('q0m0','q0m2'), ('q1m1', 'rv'), ('q0m0', 'q1m1'), ('q1m1', 'q0m2')])
cpd_q0m0 = TabularCPD(variable='q0m0',variable_card=2,values=[[0],[1]])
cpd_q1m0 = TabularCPD(variable='q1m0',variable_card=2,values=[[1],[0]])
cpd_q1m1 = TabularCPD(variable='q1m1',variable_card=2,values=[[1,0,math.cos(theta/2),-1*math.sin(theta/2)],[0,1,math.sin(theta/2),math.cos(theta/2)]],evidence=['q0m0','q1m0'],evidence_card=[2,2])
cpd_q0m2 = TabularCPD(variable='q0m2',variable_card=2,values=[[1,0,0,1],[0,1,1,0]],evidence=['q0m0','q1m1'],evidence_card=[2,2])
cpd_rv = TabularCPD(variable='rv',variable_card=2,values=[[1,0],[0,1]],evidence=['q1m1'],evidence_card=[2])

amp_damping.add_cpds(cpd_q0m0,cpd_q1m0,cpd_q0m2,cpd_q1m1,cpd_rv)
amp_damping_infer = VariableElimination(amp_damping)
amp_damping_query = amp_damping_infer.query(['rv','q0m2'])
print(amp_damping_query)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+---------+-------+----------------+
| q0m2    | rv    |   phi(q0m2,rv) |
+=========+=======+================+
| q0m2(0) | rv(0) | 0.0000+0.0000j |
+---------+-------+----------------+
| q0m2(0) | rv(1) | 0.8367+0.0000j |
+---------+-------+----------------+
| q0m2(1) | rv(0) | 0.5477+0.0000j |
+---------+-------+----------------+
| q0m2(1) | rv(1) | 0.0000+0.0000j |
+---------+-------+----------------+


In [12]:
def cpd_2_dm(obj,rvs,var):
    numQubits = len(var)
    numRVs = len(rvs)
    varOrder = obj.variables
    numVars = len(varOrder)
    qubitOrdering = []
    rvsOrdering = []
    
    for i in range(numQubits):
        v = var[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        qubitOrdering.append(2**(numVars - j - 1))
        
    for i in range(numRVs):
        v = rvs[i]
        j = 0
        while(j < numVars and v != varOrder[j]):
            j += 1
        rvsOrdering.append(2**(numVars - j - 1))

    vals = (obj.values).flatten()
    dm = np.zeros((2**numQubits,2**numQubits),dtype="complex_")
    numEvents = 2**numRVs
    numPermutations = 2**numQubits
    
    for i in range(numEvents):
        val1 = 0
        for j in range(numRVs):
            val1 += ((i//(2**j))%2)*rvsOrdering[numRVs - j - 1]
        arr1 = np.zeros((numPermutations,1),dtype="complex_")
        arr2 = np.zeros((1,numPermutations),dtype="complex_")
        for j in range(numPermutations):
            val2 = val1
            for k in range(numQubits):
                val2 += ((j//(2**k))%2)*qubitOrdering[numQubits - k - 1]
            arr1[j][0] = vals[val2]
            arr2[0][j] = np.conj(vals[val2])
        dm += np.matmul(arr1,arr2)
        
    return dm

In [29]:
X = cpd_2_dm(amp_damping_query,['rv'],['q0m2']).round(4)
print(X)

[[0.7+0.j 0. +0.j]
 [0. +0.j 0.3+0.j]]
